# <center><u> <h1>English French-Translator</h1></u></center>

Savez-vous comment créer une application de traduction de langue?<br>
Did you understand the above sentence?<br>
Well after googling it, I found its meaning as:<br>
Do you know how to create a language translator app?<br>

We all know about Google Translate which allows us to convert from one language to another and it’s very useful for learning and understanding new languages.
<br>
<br>



In this project we aim to convert English phrases to French using RNN on Deep Learning Neural Network

#Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

Preprocess - You'll convert text to sequence of integers.
Models Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
Prediction Run the model on English text.



Now let's start by importing necessary libraries.
- Importing numpy for working with arrays
-It then defines a Tokenizer object that will be used to split text into individual words using tensorflow tokenizer. 
Refer:https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
- pad_sequences is used to ensure that all sequences in a list have the same length.
Refer:https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
-Keras model represents the actual neural network model.
-the Sequential model is a linear stack of layers.
-Importing GRU,Input,Dense,TimeDistributed,Activation,RepeatVector,Bidirectional,Dropout,LSTM,Embedding from tensorflow layers.
Refer:https://www.tensorflow.org/api_docs/python/tf/keras/layers


In [1]:
#Now import above mentioned libraries 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import tensorflow as tf

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-06-10 11:30:03.589833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#Load Data
The data is located in data/small_vocab_en and data/small_vocab_fr. The small_vocab_en file contains English sentences with their French translations in the small_vocab_fr file. Load the English and French data from these files from running the cell below.

In [2]:
#load data here and split with newline to store it.

#english data
with open('small_vocab_en.txt', 'r') as eng:
    eng_data = eng.read()
eng_data = eng_data.split('\n')

#french data 
with open('small_vocab_fr.txt', 'r') as french:
    french_data = french.read()
french_data = french_data.split('\n')

In [3]:
# check few sample for English sentence and their corresponding French sentence
for i in range(5):
    print('sample:', i)
    #print english sentence
    print(eng_data[i])
    #print french sentence
    print(french_data[i])
    print('--'*50)

sample: 0
new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
----------------------------------------------------------------------------------------------------
sample: 1
the united states is usually chilly during july , and it is usually freezing in november .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
----------------------------------------------------------------------------------------------------
sample: 2
california is usually quiet during march , and it is usually hot in june .
california est généralement calme en mars , et il est généralement chaud en juin .
----------------------------------------------------------------------------------------------------
sample: 3
the united states is sometimes mild during june , and it is cold in september .
les états-unis est parfois légère en juin , et il fait froid en septembre .
---------

In [4]:
# We will remove all the ',' and '.' as our model will not be able to convert them to french
eng_data = [sentence.replace('.', '') for sentence in eng_data]
eng_data = [sentence.replace(',', '') for sentence in eng_data]
eng_data = [sentence.replace('?', '') for sentence in eng_data]

#eng_data[0:2]

#Convert to Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary. A more complex vocabulary is a more complex problem. Let's look at the complexity of the dataset we'll be working with.

A counter is a container that stores elements as dictionary keys, and their counts are stored as dictionary values.<br>
Refer:https://docs.python.org/3/library/collections.html

In [5]:
#importing collections
import collections

In [6]:
#we will check for english vocabulary 
#first we will iterate through english sentence 
#then we will split that words and then we will use counter function 
#refer more information in documentation
#we done for english vocab 
#english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()]) 
eng_word_counter = collections.Counter([word for sentence in eng_data for word in sentence.split()])
print('English Vocab:', len(eng_word_counter))
french_words_counter = collections.Counter([word for sentence in french_data for word in sentence.split()])
print('French Vocab:', len(french_words_counter))

English Vocab: 199
French Vocab: 355


#Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings. Since a neural network is a series of multiplication and addition operations, the input data needs to be numbers.

We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character. A word level model uses word ids that generate text predictions for each word. Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's Tokenizer function. Use this function to tokenize english_sentences and french_sentences in the cell below.

In [7]:
#The code tokenizes a string and returns the tokens as well as the text that was tokenized.
#define a function
def tokenize(x):
  #create a object of tokenizer
  tokenizer = Tokenizer()
  #fit the data 
  tokenizer.fit_on_texts(x)
  return tokenizer.texts_to_sequences(x), tokenizer

- The code starts by tokenizing the text_sentences list into individual sentences.Then, it prints out the word index of each sentence in the text_tokenized list.<br>
Next, it iterates through each sentence and prints out a sample output for that sentence.

In [8]:
# Tokenize Sample output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

#pass the sample text into tokenize[text_tokenized, text_tokenizer = tokenize(text_sentences)]
text_tokenized, text_tokenizer = tokenize(text_sentences)

print(text_tokenizer.word_index) 

#iterate over sample text and text_tokeinzed
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
  #print sequence
  print('sequence {} in x'.format(sample_i+1))
  #print input
  print('input {}'.format(sent))
  #print Output
  print('output {}'.format(token_sent))


{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}
sequence 1 in x
input The quick brown fox jumps over the lazy dog .
output [1, 2, 4, 5, 6, 7, 1, 8, 9]
sequence 2 in x
input By Jove , my quick study of lexicography won a prize .
output [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
sequence 3 in x
input This is a short sentence .
output [18, 19, 3, 20, 21]


#Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the end of each sequence using Keras's pad_sequences function.

In [9]:
#define a pad function and pass the x parameter 
def pad(x, length=None):
  #return pad_sequences with x and maxlength and keep padding =post
  return pad_sequences(x, maxlen=length, padding='post')

- The code is used to preprocess the input data set. - The tokenize function splits the text into individual tokens, which are then passed to a function called pad that takes in a list of tokens and pads them with a specified character (in this case, spaces).

In [10]:
#define preproces function with x and y 
def preprocess(x,y):
    #preproces the data of x
    preprocess_x, x_tk = tokenize(x)
    #preproces the data of y
    preprocess_y, y_tk = tokenize(y)

    #padding the data x
    preprocess_x = pad(preprocess_x)
    #padding the data y
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    #Expanding dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape,1)

    #return preprocess_x, preprocess_y, x_tk, y_tk
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(eng_data, french_data)

#print Max English sentence length 
max_english_sequence_length = preproc_english_sentences.shape[1]
#print Max french sentence length 
max_french_sequence_length = preproc_french_sentences.shape[1]
#print len of english vocabulary
english_vocab_size = len(english_tokenizer.word_index)
#print len of french vocabulary
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print('Max English sentence Length',max_english_sequence_length)
print('Max French Sentence Length',max_french_sequence_length)
print('English Vocab Size',english_vocab_size)
print('French Vocab Size',french_vocab_size)

Data Preprocessed
Max English sentence Length 15
Max French Sentence Length 21
English Vocab Size 199
French Vocab Size 344


#Create Model

In this section, you will experiment with various neural network architectures. You will begin by training four relatively simple architectures.

Model 1 is a simple RNN
Model 2 is a RNN with Embedding
Model 3 is a Bidirectional RNN
Model 4 is an optional Encoder-Decoder RNN
After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.

## Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want. We want the French translation. The function logits_to_text will bridge the gab between the logits from the neural network to the French translation. You'll be using this function to better understand the output of the neural network.

In [11]:
#define a function of logits to text with 2 parameters logits, tokenizer
def logits_to_text(logits, tokenizer):

  #vThe code takes in an index_to_words dictionary which maps each word to its corresponding id. - 
  #Then, it uses this dictionary to find all the words with ids 0-9 and prints them
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  #make index_to_words[0] = '<PAD>'
  index_to_words[0] = '<PAD>'

  #So basically we are predicting output for a given word and then selecting best answer
  #Then selecting that label we reverse-enumerate the word from id
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

![Model](https://github.com/tommytracey/AIND-Capstone/raw/8267d4fe72e48c595a0aff46eaf0a805fff0f36d/images/embedding.png)

#Building Model
Here we use RNN model combined with GRU nodes for translation

- The code starts by defining the input shape and output sequence length.Next, it defines the number of unique English words in the dataset and French words in the dataset.The code then builds a Keras model using word embedding on x and y.It also sets hyperparameters for learning rate, which is 0.005, as well as building layers for this model.Finally, it compiles this model with sparse_categorical_crossentropy loss function and Adam optimizer with learning rate set to 0.005. The code will create a Keras model that has been trained to recognize words in English and French.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005

    # TODO: Build the layers
    #create model Sequential 
    model = Sequential()
    #add embedding layer with english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    #add GRU layer of 256
    model.add(GRU(256, return_sequences=True))
    #add TimeDistribute layer dense of 1024 and applying of activation function of relu
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    #adding dropout layer
    model.add(Dropout(0.5))
    #add TimeDistributed Dense layer of french_vocab_size and applying softmax activation function
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    # with loss function of activation function with adam optimizer and accuaray metrix
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    #return model 
    return model
    

In [13]:
# Reshaping the input to work with a basic RNN 
#tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

Finally calling the model function

In [14]:
#calling our model 
#with tmp_x.shape, preproc_french_sentences.shape[1], len(english_tokenizer.word_index)+1,len(french_tokenizer.word_index)+1)
simple_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1
)

2024-06-10 11:31:40.168179: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-10 11:31:40.169410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-10 11:31:40.170222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Printing model summary 

In [15]:
#print Model summary 
simple_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 256)           51200     
                                                                 
 gru (GRU)                   (None, 21, 256)           394752    
                                                                 
 time_distributed (TimeDistr  (None, 21, 1024)         263168    
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 21, 1024)          0         
                                                                 
 time_distributed_1 (TimeDis  (None, 21, 345)          353625    
 tributed)                                                       
                                                                 
Total params: 1,062,745
Trainable params: 1,062,745
Non-

#Training the model
Here we start to train the model and pass the english text and the max_sequence_length, with vocab size for both english and french text

In [ ]:
#model train with tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2 
history = simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Epoch 1/20


2024-06-09 21:35:09.513214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-09 21:35:09.514664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-09 21:35:09.515567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

108/108 [==============================] - ETA: 0s - loss: 1.3578 - accuracy: 0.6817

2024-06-09 21:37:18.494571: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-09 21:37:18.495831: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-09 21:37:18.496560: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

108/108 [==============================] - 139s 1s/step - loss: 1.3578 - accuracy: 0.6817 - val_loss: 0.4976 - val_accuracy: 0.8394
Epoch 2/20
108/108 [==============================] - 134s 1s/step - loss: 0.4151 - accuracy: 0.8647 - val_loss: 0.3102 - val_accuracy: 0.8957
Epoch 3/20
108/108 [==============================] - 137s 1s/step - loss: 0.2948 - accuracy: 0.9016 - val_loss: 0.2482 - val_accuracy: 0.9158
Epoch 4/20
108/108 [==============================] - 124s 1s/step - loss: 0.2465 - accuracy: 0.9165 - val_loss: 0.2147 - val_accuracy: 0.9255
Epoch 5/20
108/108 [==============================] - 127s 1s/step - loss: 0.2204 - accuracy: 0.9243 - val_loss: 0.2035 - val_accuracy: 0.9299
Epoch 6/20
108/108 [==============================] - 138s 1s/step - loss: 0.2059 - accuracy: 0.9286 - val_loss: 0.1953 - val_accuracy: 0.9318
Epoch 7/20
108/108 [==============================] - 137s 1s/step - loss: 0.1974 - accuracy: 0.9309 - val_loss: 0.1919 - val_accuracy: 0.9333
Epoch 8/20

#Saving our model

In [ ]:
#save our model 
simple_rnn_model.save('model.h5')

In [16]:
from tensorflow.keras.models import load_model
simple_rnn_model = load_model('model.h5')

2024-06-10 11:32:53.441852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-10 11:32:53.444025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-10 11:32:53.444865: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

#Arbitrary Predictions
Performing predictions on the models using User Input.

In [17]:
#define a function for final_predictions with text timetable.
def final_predictions(text):

  #converting y_ids to words [y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}]
  y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
  #y_id_to_word[0] = '<PAD>'
  y_id_to_word[0] = '<PAD>'
  
  #spliting our english word and tokezing 
  sentence = [english_tokenizer.word_index[word] for word in text.split()]
  #padding our sentences
  sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[-2],padding='post')
  #converting into text 
  text1 = logits_to_text(simple_rnn_model.predict(sentence[:1])[0], french_tokenizer)
  #avoiding of <PAD> part in output 
  #create a variable of text 
  text2 = ''
  #iterate over text split
  for i in text1.split():
    #if i == <pad>
    if i == '<PAD>':
      #then break 
      break
    #else
    else:
      #text2=text2+" "+i
      text2 = text2+' '+i
  #return text 
  return text2


In [18]:
final_predictions(input())

2024-06-10 11:33:05.038120: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-10 11:33:05.039148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-10 11:33:05.040024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 519ms/step


' il est au printemps'

#Implementation
Enter your input here to get predictions. We will using Gradio for implementation part. Refer video for detailed information 

In [49]:
# Here we will convert and store first 100 predictions
converted_french_predictions = []
for sentence in eng_data[0:200]:
    converted_french = final_predictions(sentence)
    converted_french_predictions.append([converted_french])

1/1 [==============================] - 0s 19ms/step


In [43]:
print(converted_french_predictions[0:2],'1')
print(french_data[0:2],'1')

[[" new jersey est parfois calme en l' automne et il est neigeux en avril"], [" les états unis est généralement froid en juillet et il est généralement en l'"]] 1
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril .", 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .'] 1


In [50]:
# make same length orginal corpus
original_french = french_data[0:200]

In [51]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# List of reference translations and candidate translations
references = converted_french_predictions

candidates = original_french

# Tokenize the sentences
references = [[ref[0].split()] for ref in references]
candidates = [cand.split() for cand in candidates]

# Apply smoothing function
smoothing_function = SmoothingFunction().method1

# Compute corpus BLEU score with smoothing
bleu_score = corpus_bleu(references, candidates, smoothing_function=smoothing_function)
print(f"Corpus BLEU score: {bleu_score:.4f}")


Corpus BLEU score: 0.5647


BLEU score is decent but there are room for improvements!

In [ ]:
!pip3 install gradio 


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached jsonschema-4.22.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 1.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 1.4 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.2 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 1.6 MB

After installing, we import gradio as gr

In [52]:
import gradio as gr 

- The code creates an interface with a function called final_predictions. - The inputs of the interface are a textbox that has two lines and a placeholder, which is "Text to translate". - The outputs of the interface are "text". - The code launches the program in debug mode.

In [53]:
#creating interface for gradio 
interface = gr.Interface(fn=final_predictions,
                         inputs=gr.Textbox(lines=2, placeholder='Text to translate'),
                         outputs = 'text')

#lanuch the interface
interface.launch(debug=True)



Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 28ms/step
Keyboard interruption in main thread... closing server.


We have come to an end of this project but don't stop here, try as many projects of the similar type to get a better understanding of the use cases. 
Solve the practice sheet of this project to test yourself.!!


![](https://c.tenor.com/Irg9yU74Zo8AAAAM/woohoo-yeah.gif)

Great job!! You have come to the end of this assignment. Treat yourself for this :))
<br>
# Do fill this [feedback form](https://forms.zohopublic.in/cloudyml/form/CloudyMLDeepLearningFeedbackForm/formperma/VCFbldnXAnbcgAIl0lWv2blgHdSldheO4RfktMdgK7s)
<br>
You may head on to the next project.